In [88]:
import numpy as np
from scipy import constants
import scipy.interpolate
import math
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table
#import scipy
#from astropy.constants import h, c
%matplotlib inline


In [89]:
def mag2flux(mag, zero_pt=21.1, ABwave=None):
    if ABwave != None:
        return 10.**(-0.4*(mag+2.406+5*np.log10(ABwave)))
    return 10.**(-0.4*(mag+zero_pt))

In [90]:
def planck(wave, temp):
    w=wave/1e8
    c1 =  3.7417749e-5  #=2*!DPI*h*c*c   
    c2 = 1.4387687    # =h*c/k
    val=c2/w/temp
    bbflux=c1/(w**5 * (math.e**val-1))*1e-8
    return bbflux
#print(planck(4000, 1000))
print(range(5))

range(0, 5)


In [91]:
def gen_cal(wave, cdir, calname):
    calfile=cdir+calname
    lwave=calfile['col1']
    lflux=calfile['col2']
    caldata=scipy.interpolate.spline(lflux, lwave, wave)
    nline=len(lwave)
    nwave=len(wave)
    flux_out=wave*0.
    for i in range(nline):
        if (lwave[i] < wave[0]) or (lwave[i] > wave[-1]):
            continue
        diff=np.absolute(wave-lwave[i])
        sort_idx=np.argsort(diff)
        nrst_idx=sort_idx[0]
        binsize=wave[nrst_idx+1]-wave[nrst_idx]
        nflux=lflux[i]/binsize
        flux_out[nrst_idx]=nflux*1e-8
    return flux_out
        

In [106]:
def flux2bpmag(flam, wave, filtertrans_input, filterwave=None,
               errflag=None, flam_err=None, bpmag_err=None):
    print('?????')
    mintrans=0.0001
    filtertrans=filtertrans_input
    if filterwave != None:
        #filtertrans=interpol(filtertrans, filterwave, wave)
        filtertrans=scipy.interpolate.spline(filterwave, filtertrans, wave)
    filtertrans=(filtertrans >= mintrans)*filtertrans
    
    nwave=len(wave)
    print(len(filtertrans))
    print(nwave)
    if len(filtertrans) != nwave:
        errflag=1
        return
    
    constc=constants.c
    filtertrans=filtertrans.astype('d')
    flux=np.sum(flam*filtertrans)
#       refflam=replicate(3631.d-23, nwave)/wave^2.*!const.c*1.d10
    #refflam=replicate(3631.d-23, nwave)/wave^2.*constc*1.d10
    
    refflam=np.ones(nwave)*3631e-23/wave**2.*constc*1e10
    
    refflux=np.sum(refflam*filtertrans)
    if flam_err != None:
        err=(np.sum((flam_err*filtertrans)**2))**0.5
        bpmag_err=-2.5/np.log(10)/flux*err
    print(flux)
    print(refflux)
    bpmag=-2.5*np.log10(flux/refflux)
    return bpmag
 

In [109]:
def return_flux(source, wave, magnitude, band, z, exptime, wstep, cdir, 
                bbtemp, stype, 
                temptitle='', skyflag=0):
 #Identify source type
    consth=constants.h
    constc=constants.c
    src_check=source.split('_')
    src_type=src_check[0]
    if len(src_check) == 2:
        src_value=src_check[1]
    
    nwave=len(wave)

    if src_type =='const':
        magarr=np.ones(nwave)*magnitude
        photone=consth*1e7*constc/(wave*1e-10)
        sourceflux=mag2flux(magnitude, ABwave=wave)
        temptitle='Constant Magnitude of'
        #constflux=replicate(sourceflux[n_elements(sourceflux)/2], nwave)
        #sourceflux=constflux
    
    if src_type =='gal':
        galtempname=cdir+'kc96/'+src_value+'_template.ascii'
        galfile=ascii.read(galtempname, data_start=0)
        galwave=galfile['col1']*(1.+z)
        galflam=galfile['col2']/(1.+z)
        galflux=scipy.interpolate.spline(galflam, galwave, wave) 
        #print(galflux)
        #print(wave)
        #print(band)
        bpmag=flux2bpmag(galflux, wave, band)
        print(bpmag)
        print(galtempname)
        #print(magnitude)
        ratio=10.**(-0.4*(magnitude-bpmag))
        sourceflux=ratio*galflux
        cc=(((wave >= min(galwave)) and (wave <= max(galwave))))
        #cc=(wave >= min(galwave))
        #cc=(wave <= max(galwave))
        #print(galwave)
        #print('above galwave')
        print(cc)
        print('above cc')
        print(len(cc),len(sourceflux))
        
        sourceflux=sourceflux*((wave >= min(galwave)) and (wave <= max(galwave)))
        temptitle='Type Galaxy'
        skyflag=1
    
#    if src_type =='skyflat':

    if src_type =='arcflat':
        arcname=cdir+'Xenon_lamp.txt'
        arcfile=ascii.read(arcname, data_start=0)
        xwave=arcfile['col1']
        xflam=arcfile['col2']
        sourceflux=scipy.interpolate.spline(xflam, xwave, wave)*10.**(-12.)
        sourceflux=sourceflux*((wave >= min(xwave)) and (wave <= max(xwave)))
        temptitle='Xenon Arc Lamp'

    if src_type =='blackbody':
        sourceflux=planck(wave, bbtemp)*0.015**2.*np.pi/(1e5)**2
        temptitle='Black Body (T='

    if src_type =='wavecal':
        sourceflux=gen_cal(wave, cdir, src_value)
        temptitle='Wavelength Calibration Source - '
    
    if src_type =='sky':
        skyfilearr=['sky150701_newmoon_alt90.dat',
                    'sky160311_newmoon_alt45.dat',
                    'sky160311_halfmoon.dat',
                    'sky160311_fullmoon.dat']
        skyname=cdir+skyfilearr[stype]
        skyfile=ascii.read(skyname, data_start=37)
        skywave=skyfile['col1']*10
        skyflam=skyfile['col2']
        sourceflux=scipy.interpolate.spline(skyflam, skywave, wave)
        bpmag=flux2bpmag(galflux, wave, band)
        ratio=10.**(-0.4*(magnitude-bpmag))
        sourceflux=ratio*galflux
        sourceflux=sourceflux*((wave >= min(skywave)) and (wave <= max(skywave)))
        temptitle='Sky'

    
    return sourceflux

In [110]:
class dotifs_etc(object):
    def __init__(self, exptime=900, band='r', magnitude=17., 
                 oname='snr.ps', galtemp=False, z=0., inputflux=None, inputwave=None, 
                 #bbtemp=False, wavecal=False, skyflat=False, arcflat=False,
                 source='gal_sc', bbtemp=5000, scflag=False, 
                 stype=0, skymagnitude=22., nofilter=False,
                 noplot=False, plotrange=[3700, 7400], wstep=(3700./3000), pixel=None, 
                 cspline=False, wavearr=None,
                 waveout=None, snrout=None, signalarr=None, skysignalarr=None, noisearr=None, noiseskyarr=None, 
                 dtypes=None, ofile='outdata.txt', 
                 pri=3.6, sec=0.915, skysamplingsize=0.4**2*math.pi, dispersion=3700/(3000*15), pixelsize=15,
                 npix_spa=5, 
                 cdir='/home/hchung/dotifs/py_etc/',
                ):
        use_asahi=1
        ltrghost=1
        consth=constants.h
        constc=constants.c
        temptitle='None'
        
        ifutrans=0.85*0.9
        telaream2=(pri**2-sec**2)/4*math.pi         #in m^2
        telarea=telaream2*1e4                           #in cm^2
        pixelscale=dispersion*pixelsize
        if pixel != None:
            wstep=pixel*pixelscale
        stwave=plotrange[0]
        edwave=plotrange[1]
        
        
        transfname=cdir+'trans150626.dat'
        #data=ascii.read(transfile, format='no_header')
        transfile=ascii.read(transfname, data_start=1)
        wavemicron=np.array(transfile['micron'])
        waveang=wavemicron*1e4
        skytrans=np.array(transfile['SkyTrans'])
        telmag=np.array(transfile['telandmag'])
        col=np.array(transfile['col'])
        cam=np.array(transfile['cam'])
#        ccd=np.array(transfile['ccd'])
        g0th_o=np.array(transfile['0th'])
        g1st_o=np.array(transfile['1st'])
        g2nd_o=np.array(transfile['2nd'])
        
        vphfname=cdir+'vph_160307_new.dat'
        vphfile=ascii.read(vphfname, data_start=1)

        coatfname=cdir+'altcoating.dat'
        coatfile=ascii.read(coatfname, data_start=1)     
        
        ccdfname=cdir+'ccd_multi2.txt'
        ccdfile=ascii.read(ccdfname, data_start=1)

        bandname=cdir+band+'filter.dat'
        bandfile=ascii.read(bandname, data_start=2)
        
        afiltername=cdir+'asahi_filter_01.dat'
        afilterfile=ascii.read(afiltername, data_start=0)
        
        
        if wavearr == None:
            nwave=int((waveang[-1]-waveang[0])/wstep)+1
            wave=np.linspace(0, nwave-1, num=nwave)*wstep+waveang[0]
        else:
            wave=np.array(wavearr)
            nwave=len(wavearr)
        print('wavelen ', len(wave))
        
        if cspline:
            eff_vph_new=scipy.interpolate.spline(vphfile['wave_new'],
                                                 vphfile['vph_trans'],
                                                 waveang/10)
            eff_coat=scipy.interpolate.spline(coatfile['wavenm'],
                                              coatfile['t0'],
                                              waveang/10)/100
            ccd=scipy.interpolate.spline(ccdfile['Wave'],
                                         ccdfile['QE'],
                                         waveang/10)
            band=scipy.interpolate.spline(bandfile['col1'],
                                          bandfile['col2'],
                                          wave)
            afilter=scipy.interpolate.spline(afilterfile['col1']*10,
                                             afilterfile['col2']/100,
                                             waveang)
        else:
            eff_vph_new=np.interp(waveang/10,vphfile['wave_new'],vphfile['vph_trans'])
            eff_coat=np.interp(waveang/10, coatfile['wavenm'], coatfile['t0'])/100
            ccd=np.interp(waveang/10,ccdfile['Wave'],ccdfile['QE'])
            band=np.interp(wave, bandfile['col1'], bandfile['col2'])
            afilter=np.interp(waveang, afilterfile['col1']*10,afilterfile['col2']/100)
            
        print('waveang?',len(waveang))
        
        g1st=eff_vph_new
        
        #waveang=tfile['micron']*1e4
        

            
        skyflag=0
        photone=consth*1e7*constc/(wave*1e-10)
        sourceflux=return_flux(source, wave, magnitude, band, z, 
                               exptime, wstep, cdir, bbtemp, stype,
                               temptitle=temptitle, skyflag=skyflag)
            
        sourcecount=(sourceflux/photone)*wstep*telarea*exptime*skysamplingsize
    
        comtrans=telmag*ifutrans*col*cam*ccd
        t1st=comtrans*g1st
        t2nd=comtrans*g2nd*0.5   
            #light from short wavelength are divided into double wavelength bin
        tsky=skytrans
        if cspline:
            t1st=scipy.interpolate.spline(t1st, waveang, wave)
            t2nd=scipy.interpolate.spline(t2nd, waveang, wave)
            tsky=scipy.interpolate.spline(tsky, waveang, wave)
        else:
            t1st=np.interp(wave, t1st, waveang)
            t2nd=np.interp(wave, t2nd, waveang)
            tsky=np.interp(wave, tsky, waveang)
        
        t1st=t1st*afilter
        t2nd=t2nd*afilter
        tsky=tsky*afilter
        
        pc1st=t1st*sourcecount
        pc2nd=t2nd*sourcecount
    
        if skyflaq == 1:
            pc1st=pc1st*tsky
            pc2nd=pc2nd*tsky
        
        skypc1st=t1st*skycount
        skypc2nd=t2nd*skycount

        ########
        wave2nd=wave*2
        if cspline:
            pc2nd=scipy.interpolate.spline(pc2nd, wave2nd, wave)
            skypc2nd=scipy.interpolate.spline(skypc2nd, wave2nd, wave)
        else:
            pc2nd=np.interp(wave, pc2nd, wave2nd)
            skypc2nd=np.interp(wave, skypc2nd, wave2nd)
            
        pc2nd=pc2nd*((wave >= min(wave2nd)) and (wave <= max(wave2nd)))
        skypc2nd=skypc2nd*((wave >= min(wave2nd)) and (wave <= max(wave2nd)))
        if scflag == False:
            pc2nd=pc2nd*0.
            skypc2nd=skypc2nd*0.

        rn_t=rn*(npix_spa*pixel)**0.5
        dark_t=dark*exptime/3600.*(npix_spa*pixel)**0.5
#;print, dark, exptime, npix_spa, pixel, dark_t, rn_t
#;       print, rn_t


        signal=pc1st+pc2nd
#;       signal=pc1st
        skysignal=skypc1st+skypc2nd

        if ltrghost !=False:
            lghost=return_littrow_ghost(wave,signal+skysignal, ccdreflect, cam, g1stR, g1st, g0th)
            signal=signal+lghost

#;print, dark, exptime, npix_spa, pixel, dark_t, rn_t, mean((signal+2*skysignal)^0.5)
        noise_poisson=(signal+skysignal+rn_t**2+dark_t**2)**0.5
        noise_sky=(skysignal+rn_t**2+dark_t**2)**0.5
        noise_2nd=pc2nd
        noise=(noise_poisson**2+noise_sky**2)**0.5
#;       noise_total=noise_poisson+noise_sky
        nfrac_poisson=signal/noise
        nfrac_sky=skysignal*2/noise
        nfrac_rn=2**0.5*rn_t/noise
        nfrac_dark=2**0.5*dark_t/noise
        snr=signal/noise
        pc2vsntotal=pc2nd/noise
        
        #########
        idx=np.nonzero((wave >= stwave) and (wave <= edwave))
        psourceflux=sourceflux[idx]
        pwave=wave[idx]
        psnr=snr[idx]
        psignal=signal[idx]
        pskysignal=skysignal[idx]
        pnoise_sky=noise_sky[idx]
        pnoise=noise[idx]
        pnfrac_poisson=nfrac_poisson[idx]
        pnfrac_sky=nfrac_sky[idx]
        ppc2vsntotal=pc2vsntotal[idx]
        ppc2nd=pc2nd[idx]
        pskyfrac=nfrac_sky[idx]
        prnfrac=nfrac_rn[idx]
        waveout=pwave

        snrout=psnr
        signalarr=psignal
        skysignalarr=pskysignal
        noisearr=pnoise
        noiseskyarr=pnoise_sky
        skyfrac=pskyfrac
        readnoisefrac=prnfrac
        sourcefluxout=psourceflux

        
        #signal=sourcecount
        #oise=signal**0.5

        
        #snr=signal/noise
#        print(snr)

        signal=psignal
        noise=pnoise
        snr=psnr
        
        matplotlib.rcParams.update({'font.size':15})
        font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 15}

        matplotlib.rc('font', **font)
        
        plt.clf
        fig=plt.figure(figsize=(7,9))
        #gs1=matplotlib.gridspec.GridSpec(8,12)
        gs1=gridspec.GridSpec(4,1)
        gs1.tight_layout(fig, rect=[0,0,0.1,1])
        gs1.update(hspace=0)
        #gs1.tight_layout(fig, rect=[0,0,1,0.5])
        
        
        #ax=plt.subplot(211)
        ax0=plt.subplot(gs1[0])
        ax0.plot(wave, sourceflux, 'k', linestyle='-')
        plt.text(5000,0.00000001*0, 'matplot')
        #ax=plt.subplot(212)
        ax=plt.subplot(gs1[1])
        ax.plot(wave, snr, 'rh')
        #ax.set_ylabel("S/N", fontsize=20)
        #ax.set_xlabel("Wavelength $\AA$ ", fontsize=20)
        ax.set_ylabel("S/N")
        ax.set_xlabel("Wavelength ($\AA$) ")
        ax.grid(b=True, which='major', color='k', linestyle='--', linewidth=1)
        ax.xaxis.set_minor_locator(plt.MultipleLocator(200))
        #ax.xlim([2000,10000])
        xlim=[2000,10000]
        ylim=[0,199]
        
        ax.set_xlim(xlim)
        ax0.grid(b=True, which='major', color='r', linestyle='--', linewidth=1)
        ax0.set_xlim(xlim)
        ax.set_ylim(ylim)
        
        '''
        plt.subplot(231)
        plt.subplot(236)
        plt.subplot(235)
        plt.subplot(234)
        '''
        
        ax2=plt.subplot(gs1[2])
        ax2.plot(wave, sourceflux, 'k', linestyle='-')
        
        ax3=plt.subplot(gs1[3])
        ax3.plot(wave, sourceflux, 'k', linestyle='-')
        
        #plt.text(0.5, 1.2, 'matplotlib', horizontalalignment='center',
        #         verticalalignment='center', transform=ax.transAxes)
        
        #plt.scatter(wave, snr)
        
        
        
def codetest():
    dotifs_etc(300)
#!cat ~/dotifs_etc/trans150626.dat
codetest()

wavelen  3487
waveang? 861


/home/hchung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: `spline` is deprecated!
spline is deprecated in scipy 0.19.0, use Bspline class instead.


?????
3487
3487
0.0
2.9976904230344304e-06
inf
/home/hchung/dotifs/py_etc/kc96/sc_template.ascii


/home/hchung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10
/home/hchung/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in multiply


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import numpy as np
arr=[0,1,2,3,4]
arr=np.array(arr)
print(len(arr))